### Installing Requirements and importing libraries

In [1]:
pip install -r requirements.txt

  Using cached selenium-4.17.2-py3-none-any.whl (9.9 MB)
  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
  Using cached trio-0.24.0-py3-none-any.whl (460 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import csv

Scraping the 224 individual refresher readings links

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
total=0
page_link = []

def scrape_links(driver):
    n=0
    global total
    global page_link
    links = driver.find_elements(By.CSS_SELECTOR, 'div.cfa-coveo a')
    for link in links:
        href = link.get_attribute('href')
        if href is not None:
            n=n+1
            page_link.append(href)
            print(href)
        
    total=total+n
    
def go_to_next_page(driver):
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "li.coveo-pager-next")
        
        driver.execute_script("arguments[0].click();", next_button)
        return True
    except NoSuchElementException:
        return False

url = 'https://www.cfainstitute.org/en/membership/professional-development/refresher-readings'
driver.get(url)


while True:
    
    try:
        WebDriverWait(driver, 600).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.cfa-coveo')))
        scrape_links(driver)
    except TimeoutException:
        print("Timed out waiting for page to load")
        break  
    if not go_to_next_page(driver):
        print("No more pages or next button not found.")
        break
    time.sleep(5)





urls = page_link.copy()

https://www.cfainstitute.org/membership/professional-development/refresher-readings/time-series-analysis
https://www.cfainstitute.org/membership/professional-development/refresher-readings/credit-analysis-models
https://www.cfainstitute.org/membership/professional-development/refresher-readings/introduction-alternative-investments
https://www.cfainstitute.org/membership/professional-development/refresher-readings/credit-default-swaps
https://www.cfainstitute.org/membership/professional-development/refresher-readings/valuation-contingent-claims
https://www.cfainstitute.org/membership/professional-development/refresher-readings/introduction-commodities-commodity-derivatives
https://www.cfainstitute.org/membership/professional-development/refresher-readings/understanding-income-statements
https://www.cfainstitute.org/membership/professional-development/refresher-readings/pricing-and-valuation-of-forward-commitments
https://www.cfainstitute.org/membership/professional-development/refresher

https://www.cfainstitute.org/membership/professional-development/refresher-readings/principles-asset-allocation
https://www.cfainstitute.org/membership/professional-development/refresher-readings/technical-analysis
https://www.cfainstitute.org/membership/professional-development/refresher-readings/introduction-risk-management
https://www.cfainstitute.org/membership/professional-development/refresher-readings/portfolio-risk-return-part-2
https://www.cfainstitute.org/membership/professional-development/refresher-readings/portfolio-risk-return-part-1
https://www.cfainstitute.org/membership/professional-development/refresher-readings/economics-investment-markets
https://www.cfainstitute.org/membership/professional-development/refresher-readings/swaps-forwards-futures-strategies
https://www.cfainstitute.org/membership/professional-development/refresher-readings/overview-equity-portfolio-management
https://www.cfainstitute.org/membership/professional-development/refresher-readings/options-st

https://www.cfainstitute.org/membership/professional-development/refresher-readings/forward-commitment-contingent-claim-features-instruments
https://www.cfainstitute.org/membership/professional-development/refresher-readings/pricing-valuation-futures-contracts
https://www.cfainstitute.org/membership/professional-development/refresher-readings/business-models-risks
https://www.cfainstitute.org/membership/professional-development/refresher-readings/capital-investments
https://www.cfainstitute.org/membership/professional-development/refresher-readings/Interest-Rate-Risk-and-Return
https://www.cfainstitute.org/membership/professional-development/refresher-readings/Yield-Based-Bond-Duration-Measures-and-Properties
https://www.cfainstitute.org/membership/professional-development/refresher-readings/introduction-geopolitics
https://www.cfainstitute.org/membership/professional-development/refresher-readings/Yield-and-Yield-Spread-Measures-for-Floating-Rate-Instruments
https://www.cfainstitute.o

Extracting the data individual 224 webpages and storing it CSV File

In [7]:


csv_file_path = 'extracted.csv'

# CSV headers
headers = ['Name_of_the_topic', 'Year', 'Level', 'Introduction_Summary', 'Learning_Outcomes', 'Link_to_the_Summary_Page', 'Link_to_the_PDF_File']

def safe_extract_text(soup, selector, attribute=None):
    element = soup.select_one(selector)
    if attribute:
        return element[attribute] if element and attribute in element.attrs else "None"
    return element.get_text(strip=True) if element else "None"

def extract_following_text(header):
    if header:
        content = []
        for sibling in header.find_next_siblings():
            if sibling.name == "h2":
                break  # Stop if we encounter another header
            content.append(sibling.get_text(strip=True))
        return " ".join(content)
    return "None"

# Change to 'w' mode to overwrite the file and include headers at the beginning
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    
    for url in urls:
        # Fetch the HTML content of the page
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        
        curriculum_year = safe_extract_text(soup, ".content-utility-curriculum")
        level = safe_extract_text(soup, ".content-utility-level")
        topic = safe_extract_text(soup, ".content-utility-topic")
        pdf_link = safe_extract_text(soup, "a.locked-content", "href")
        
        
        introduction_header = soup.find("h2", text="Introduction")
        introduction_text = extract_following_text(introduction_header)
        
        learning_outcomes_header = soup.find("h2", text="Learning Outcomes")
        learning_outcomes_text = extract_following_text(learning_outcomes_header)
        
        # Write the extracted data to the CSV
        data = {
            "Name_of_the_topic": topic,
            "Year": curriculum_year,
            "Level": level,
            "Introduction_Summary": introduction_text,
            "Learning_Outcomes": learning_outcomes_text,
            "Link_to_the_Summary_Page": url,
            "Link_to_the_PDF_File": pdf_link
        }
        
        writer.writerow(data)